In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
keys = ['时间', '水温', 'pH', '溶解氧', '电导率', '浊度', '高锰酸盐指数', '氨氮', '总磷', 
            '总氮', '湿度', '室温', '叶绿素', '藻密度']
data = pd.read_csv('./data.csv')
data['时间'] = pd.to_datetime(arg=data['时间'], format='%Y-%m-%d %H:%M:%S')
# data = data.set_index(data.columns[1])
data

,时间,水温,pH,溶解氧,电导率,浊度,高锰酸盐指数,氨氮,总磷,总氮
0,2018-07-26 00:00:00,20.332,7.552,3.017,239.618,2.198,0.950,0.070,0.007,0.065
1,2018-07-26 04:00:00,20.310,7.573,3.072,238.424,2.150,0.740,0.070,0.000,0.000
2,2018-07-26 07:00:00,20.305,7.537,2.999,238.001,2.099,0.760,0.080,0.000,0.000
3,2018-07-26 12:00:00,20.377,7.521,3.116,239.050,2.052,0.640,0.080,0.006,0.046
4,2018-07-26 16:00:00,20.492,7.582,3.639,238.861,2.110,0.660,0.080,0.007,0.058
...,...,...,...,...,...,...,...,...,...,...
8107,2022-09-22 04:00:00,19.755,8.070,6.169,241.515,4.953,0.799,0.005,0.006,0.251
8108,2022-09-22 08:00:00,19.716,8.060,6.211,240.581,4.459,0.817,0.005,0.003,0.234
8109,2022-09-22 12:00:00,19.720,8.060,6.076,241.125,4.720,0.719,0.006,0.006,0.241
8110,2022-09-22 16:00:00,19.764,8.030,6.158,240.930,5.518,0.954,0.005,0.004,0.256


In [24]:
DFList = []
keys = data.keys()
print(list(keys))
for group in data.groupby([data['时间'].dt.year, data['时间'].dt.month, data['时间'].dt.day]):
    if group[1].shape[0] == 6:
        df = group[1][::-1].values.copy()
        DFList.append(df)
arr = np.concatenate(DFList, axis=0)
arr.shape
new_data = pd.DataFrame(arr, columns=keys)
new_data.to_csv('./data.csv', index=False)

['时间', '水温', 'pH', '溶解氧', '电导率', '浊度', '高锰酸盐指数', '氨氮', '总磷', '总氮']


In [3]:
# insert middle values by the mean values
keys = data.keys()[1:]
for k in keys:
    for i in range(1, len(data[k]) - 1):
        val0 = data[k][i]
        val1 = data[k][i-1]
        val2 = data[k][i+1]
        
        if np.isnan(val0) and not np.isnan(val1) and not np.isnan(val2):
            print(f'{k} {i}')
            data[k][i] = (val1 + val2)/2


In [13]:
from scipy.stats import zscore
# Data value clean
# Methods:
#     Numeric Outlier --> 通过计算第一和第三分位数　小于和大于其二的去掉
#     Z-score --> 计算此数据点和总数据标准差的比，
def detect_outlier(df, label, rate=4):
    dfvals = df[label]
    indices = np.array(list(map(lambda x: not x, np.isnan(dfvals))))
    values = dfvals[indices]
    Q1 = np.percentile(values, rate)
    Q3 = np.percentile(values, 100-rate)    
    
    IQR = Q3 - Q1
    
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    outlier_list = dfvals[(dfvals < lower_limit) | (dfvals > upper_limit)].index
    return outlier_list

def compute_zscore(df, label, threshold=2.5):
    data = df[label]
    all_values = data.values.copy()
    indices = np.array(list(map(lambda x: not x, np.isnan(all_values))))
    values = all_values[indices]
    z_value = zscore(values)
    all_values[indices] = z_value
    ls = data[np.abs(all_values) > threshold].index
    return ls

def myFilter(df, labels = ['电导率', '高锰酸盐指数'], name='./data_filtered.csv'):
    dfn = df.copy()
    for label in labels:
        dfvals = dfn[label].values.copy()
        out_list = compute_zscore(dfn, label, 1.5)
        dfvals[out_list] = None
        dfn[label] = dfvals
    dfn.to_csv(name)
    return True

True

In [3]:
# # Data type clean
# original_data = pd.read_excel('./original_data.xls', header=2)
# # print(original_data.keys())
# data = pd.DataFrame()
# ls = [pd.to_datetime(arg=t, format='%Y-%m-%d %H:%M:%S') for t in original_data['采集时间']]
# data['时间'] = ls
# for i in range(1, len(keys)):
#     ls = []
#     for val in original_data[keys[i]]:
#         if val == '--':
#             fval = None
#         else:
#             fval = np.abs(float(val))
#         ls.append(fval)
#     data[keys[i]] = ls
# data.to_csv('data.csv', mode='a')
# values = data['pH']
# values[0]
# values = list(filter(lambda x: x>0, values))